In [2]:
import os
import json
import pandas as pd
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
results_dir = "/content/drive/Shareddrives/CS198-Drones/[v4] FFT Module Evaluation"
results_output_file = "/content/drive/Shareddrives/CS198-Drones/[v4] FFT Module Evaluation/evaluation.xlsx"

training_dir = "/content/drive/Shareddrives/CS198-Drones/[v4] Training Output/Full Fine-tune (base)"
training_output_file = "/content/drive/Shareddrives/CS198-Drones/[v4] Training Output/Full Fine-tune (base)/training_metrics.xlsx"

In [8]:
def compile_evaluation_results(json_folder, output_file):
    precision_data = []
    recall_data = []
    f1_data = []
    overall_data = []

    for model_dir in os.listdir(json_folder):
        model_path = os.path.join(json_folder, model_dir)
        json_file_path = os.path.join(model_path, "report.json")
        if os.path.isdir(model_path) and os.path.isfile(json_file_path):
            with open(json_file_path, "r") as f:
                data = json.load(f)
                model_name = model_dir

                for class_name, metrics in data.items():
                    if isinstance(metrics, dict):
                        precision_data.append({"Model": model_name, "Class": class_name, "Precision": metrics.get("precision", None)})
                        recall_data.append({"Model": model_name, "Class": class_name, "Recall": metrics.get("recall", None)})
                        f1_data.append({"Model": model_name, "Class": class_name, "F1-Score": metrics.get("f1-score", None)})

                # Add overall data
                weighted_ave = data["weighted avg"]
                overall_data.append({"Model": model_name, "Accuracy": data.get("accuracy", None), "Precision": weighted_ave.get("precision", None), "Recall": weighted_ave.get("recall", None), "F1-Score": weighted_ave.get("f1-score", None), "Support": weighted_ave.get("support", None), "Eval Time": data.get("evaluation_time_sec", None)})

    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        pd.DataFrame(precision_data).pivot(index="Model", columns="Class", values="Precision").to_excel(writer, sheet_name="Precision")
        pd.DataFrame(recall_data).pivot(index="Model", columns="Class", values="Recall").to_excel(writer, sheet_name="Recall")
        pd.DataFrame(f1_data).pivot(index="Model", columns="Class", values="F1-Score").to_excel(writer, sheet_name="F1-Score")
        pd.DataFrame(overall_data).to_excel(writer, sheet_name="Overall Accuracy", index=False)

    print(f"Results saved to {output_file}")

In [14]:
def compile_training_metrics(metrics_folder, output_file):
    # Define metric categories (these will be used as sheet names)
    metric_names = [
        "loss", "grad_norm", "learning_rate", "eval_loss", "eval_accuracy",
        "eval_runtime", "eval_samples_per_second", "eval_steps_per_second",
        "gpu_vram_allocated_mb", "gpu_vram_reserved_mb"
    ]

    # Initialize a dictionary to hold DataFrames for each metric
    metrics_dict = {metric: pd.DataFrame() for metric in metric_names}

    # Loop through model folders
    for model_dir in os.listdir(metrics_folder):
        model_path = os.path.join(metrics_folder, model_dir)
        metrics_file = os.path.join(model_path, "training_metrics.xlsx")

        if os.path.isdir(model_path) and os.path.isfile(metrics_file):
            df = pd.read_excel(metrics_file)

            if "epoch" not in df.columns:
                print(f"Skipping {model_dir} (No 'epoch' column found)")
                continue

            # Ensure epoch is integer and sort
            df["epoch"] = df["epoch"]

            # Add each metric to its respective dictionary entry
            for metric in metric_names:
                if metric in df.columns:
                    if metrics_dict[metric].empty:
                        metrics_dict[metric] = df[["epoch", metric]].copy()
                        metrics_dict[metric].rename(columns={metric: model_dir}, inplace=True)
                    else:
                        metrics_dict[metric] = pd.merge(metrics_dict[metric], df[["epoch", metric]], on="epoch", how="outer")
                        metrics_dict[metric].rename(columns={metric: model_dir}, inplace=True)

    # Save to Excel
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for metric, df in metrics_dict.items():
            if not df.empty:
                df.to_excel(writer, sheet_name=metric, index=False)

    print(f"Training metrics compiled and saved to {output_file}")

In [7]:
compile_evaluation_results(results_dir, results_output_file)

Results saved to /content/drive/Shareddrives/CS198-Drones/[v4] FFT Module Evaluation/evaluation.xlsx


In [15]:
compile_training_metrics(training_dir, training_output_file)

Training metrics compiled and saved to /content/drive/Shareddrives/CS198-Drones/[v4] Training Output/Full Fine-tune (base)/training_metrics.xlsx
